In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
import optuna
import optuna.visualization as vis

# ==========================
# 【1. CSVファイルの読み込みとデータ準備】
# カレントディレクトリ内の全てのCSVファイルを結合し、必要な列を抽出して学習データを作成します。
# ==========================
csv_files = glob.glob("*.csv")
dfs = [pd.read_csv(f) for f in csv_files]
data = pd.concat(dfs, ignore_index=True)
data = data.dropna(subset=[data.columns[4]])  # 目的変数列

X = data.iloc[:, :4].values  # Pressure, Gas flow, Line, Space
y = data.iloc[:, 4].values   # Amount of SiO2 etched

# ==========================
# 【2. 特徴量と目的変数の標準化】
# モデル学習の安定化のため、入力（X）と出力（y）を標準化します。
# ==========================
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.reshape(-1,1)).ravel()

# ==========================
# 【3. ガウス過程回帰（GPR）モデルの作成】
# カーネルを指定し、標準化済みデータでGPRモデルを学習します。
# ==========================
def create_gpr():
    kernel = ConstantKernel(1.0) * Matern(length_scale=np.ones(X.shape[1]), nu=2.5) + WhiteKernel(noise_level=1)
    gpr = GaussianProcessRegressor(kernel=kernel, normalize_y=True, random_state=42, n_restarts_optimizer=5)
    gpr.fit(X_scaled, y_scaled)
    return gpr

gpr = create_gpr()

# ==========================
# 【4. Optunaを用いたベイズ最適化】
# GPRモデルを用いて、SiO2エッチ量が最大となる条件（4変数）をOptunaで探索します。
# ==========================
def objective(trial):
    pressure = trial.suggest_float("Pressure", X[:,0].min(), X[:,0].max())
    gasflow  = trial.suggest_float("GasFlow", X[:,1].min(), X[:,1].max())
    line     = trial.suggest_float("Line", X[:,2].min(), X[:,2].max())
    space    = trial.suggest_float("Space", X[:,3].min(), X[:,3].max())

    x_trial = np.array([[pressure, gasflow, line, space]])
    x_trial_scaled = scaler_X.transform(x_trial)

    y_pred_scaled = gpr.predict(x_trial_scaled)
    y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1,1))[0,0]
    return y_pred

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("Best parameters:", study.best_params)
print("Best predicted SiO2 Etch:", study.best_value)

# ==========================
# 【5. Optunaによる最適化履歴の可視化】
# 最適化の過程（各試行での値の推移）をPlotlyグラフで表示・保存します。
# ==========================
fig_history = vis.plot_optimization_history(study, target=lambda t: t.values[0], target_name="SiO2 Etched (nm)")
fig_history.update_layout(width=800, height=600)
fig_history.write_html("optimization_history.html")  # HTML保存
fig_history.show()

# ==========================
# 【6. 2次元ヒートマップの作成と最適化履歴の重ね描き】
# PressureとGasFlowを変化させ、Line/Spaceを中央値で固定して、GPRモデルの予測値のヒートマップを描きます。
# Optunaによる探索点も同じ図上にプロットします。
# ==========================
line_med = np.median(X[:,2])
space_med = np.median(X[:,3])

# PressureとGasFlowのグリッドを生成
pressure_grid = np.linspace(0, 4.0, 50)   # Pressure: 0～4.0
gasflow_grid  = np.linspace(0, 250, 50)   # Gas Flow: 0～250
P, G = np.meshgrid(pressure_grid, gasflow_grid)

grid_points = np.column_stack([
    P.ravel(),
    G.ravel(),
    np.full(P.size, line_med),
    np.full(P.size, space_med)
])
grid_points_scaled = scaler_X.transform(grid_points)
Z_scaled, std_scaled = gpr.predict(grid_points_scaled, return_std=True)
Z = scaler_y.inverse_transform(Z_scaled.reshape(-1,1)).reshape(P.shape)

plt.figure(figsize=(8,6))
cp = plt.contourf(P, G, Z, levels=50, cmap='viridis')
plt.colorbar(cp, label='Predicted SiO2 Etch (nm)')

# ==========================
# 【7. Optuna探索履歴の点をヒートマップ上にプロット】
# ==========================
trial_P = [t.params["Pressure"] for t in study.trials if t.params]
trial_G = [t.params["GasFlow"] for t in study.trials if t.params]
if trial_P and trial_G:
    plt.scatter(trial_P, trial_G, c='black', s=30, marker='o', label='Optimization Trial')

plt.xlabel('Pressure (Pa)')
plt.ylabel('Gas flow (sccm)')
plt.title(f'2D Heatmap with Optimization Trials\n(Line={line_med}, Space={space_med})')
plt.legend()
plt.tight_layout()
plt.savefig("heatmap_with_trials.png")
plt.show()